# Defining Neural Network Architecture


## Architecture :
- 784 neurons (input layer) + 1 bias and relu activation
- 16 neurons + 1 bias and relu activation
- 16 neurons + 1 bias and relu activation
- 10 neurons (output layer) and softmax activation



In [1]:
import numpy as np

In [ ]:
class DNN:
    def __init__(self, learning_rate, input_layer, hidden_layer_1, hidden_layer_2, output_layer):
        self.learning_rate = learning_rate
        self.a1 = np.zeros((input_layer, 1))
        self.a2 = np.zeros((hidden_layer_1, 1))
        self.a3 = np.zeros((hidden_layer_2, 1))
        self.a4 = np.zeros((output_layer, 1))

        self.weight_1 = np.random.randn(16, 784)
        self.weight_2 = np.random.randn(16, 16)
        self.weight_3 = np.random.randn(10, 16)

        self.bias_1 = np.random.randn(16, 1)
        self.bias_2 = np.random.randn(16, 1)
        self.bias_3 = np.random.randn(10, 1)
    
    def relu(self, inp):
        a = []
        for i in inp:
            b = max(i, 0)
            if(b == 0):
                b = np.array([0])
            a.append(b)
        return(np.array(a))
    
    def softmax(self, inp):
        denominator = np.sum(np.exp(inp))
        a = []
        for i in inp:
            numerator = np.exp(i)
            probability = numerator/denominator
            a.append(probability)
        return(np.array(a))
    
    def forwardpropogation(self, x:np.array):
        self.a1 = self.relu(np.reshape(x, (x.shape[0], 1)))

        z2 = np.matmul(self.weight_1, self.a1) + self.bias_1
        self.a2 = self.relu(z2)

        z3 = np.matmul(self.weight_2, self.a2) + self.bias_2
        self.a3 = self.relu(z3)

        z4 = np.matmul(self.weight_3, self.a3) + self.bias_3
        self.a4 = self.softmax(z4)
        return(self.a4)